In [27]:
import torch
import requests
import clip
import numpy as np
from PIL import Image
import requests


def do_clip(url: str, text_snippets: list[str]) -> None:

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load("ViT-B/32", device=device)

    text = clip.tokenize(text_snippets).to(device)
    image_input = Image.open(requests.get(url, stream=True).raw)
    image_input = preprocess(image_input).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image_input)
        logits_per_image, _ = model(image_input, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    print("\nTop predictions:\n")
    for value, index in zip(probs[0], text_snippets):
        print(f"{index}: {100 * value.item():.2f}%")


# avengers picture
url = 'https://cdn.kanobu.ru/games/f8ffb106-1d6b-497c-8b12-3943d570ddc3.jpg'
text_snippets = ["hulk", "iron men", "black widow"]

do_clip(url, text_snippets)


Top predictions:

hulk: 8.72%
iron men: 90.58%
black widow: 0.70%
